<a href="https://colab.research.google.com/github/Rohithtechis/Logistic-regression/blob/main/Bagging_boosting_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import zscore

In [81]:
data=pd.read_csv('/content/talking_data.csv')
data.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,NaN,0
1,105560,25,1,17,259,2017-11-07 13:40:27,NaN,0
2,101424,12,1,19,212,2017-11-07 18:05:24,NaN,0
3,94584,13,1,13,477,2017-11-07 04:58:08,NaN,0
4,68413,12,1,1,178,2017-11-09 09:00:09,NaN,0


In [82]:
data.shape

(100000, 8)

In [83]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   ip               100000 non-null  int64 
 1   app              100000 non-null  int64 
 2   device           100000 non-null  int64 
 3   os               100000 non-null  int64 
 4   channel          100000 non-null  int64 
 5   click_time       100000 non-null  object
 6   attributed_time  227 non-null     object
 7   is_attributed    100000 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 6.1+ MB


In [84]:
data.describe()

,ip,app,device,os,channel,is_attributed
count,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000
mean,91255.879670,12.04788,21.771250,22.818280,268.832460,0.002270
std,69835.553661,14.94150,259.667767,55.943136,129.724248,0.047591
min,9.000000,1.00000,0.000000,0.000000,3.000000,0.000000
25%,40552.000000,3.00000,1.000000,13.000000,145.000000,0.000000
50%,79827.000000,12.00000,1.000000,18.000000,258.000000,0.000000
75%,118252.000000,15.00000,1.000000,19.000000,379.000000,0.000000
max,364757.000000,551.00000,3867.000000,866.000000,498.000000,1.000000


1. Explore the dataset for anomalies and missing values and take corrective actions if necessary.

In [85]:
data.isna().sum()

ip                     0
app                    0
device                 0
os                     0
channel                0
click_time             0
attributed_time    99773
is_attributed          0
dtype: int64

According to the info attribute_time is the time of the app downloaded by clicking the ad, so there are 99773 missing value this means they all are related to fraud clicks.

2. Which column has maximum number of unique values present among all the available columns

In [86]:
data.columns.unique()

Index(['ip', 'app', 'device', 'os', 'channel', 'click_time', 'attributed_time',
       'is_attributed'],
      dtype='object')

In [87]:
len(data.ip.unique())

34857

In [88]:
len(data.app.unique())

161

In [89]:
len(data.device.unique())

100

In [90]:
len(data.os.unique())

130

In [91]:
len(data.channel.unique())

161

In [92]:
len(data.click_time.unique())

80350

In [93]:
len(data.attributed_time.unique())

228

In [94]:
len(data.is_attributed.unique())

2

We can conclude that click_time has the most number of unique values

Use an appropriate technique to get rid of all the apps that are very rare (say which comprise of less than 20% clicks) and plot the rest.

In [95]:
data.app.unique()

array([ 12,  25,  13,   3,   1,   9,   2,   6,  20,  14,  64,   7,  15,
        18,  11,   8,  23,  21,  56,  28,  36,  22,  26,  17,  27,  10,
        60,  58,  19,  32,  24,   4,  53, 125, 204,   5, 202,  70,  47,
        29,  55, 103,  35,  83, 208, 146, 161,  46,  62,  65, 110,  42,
       150, 266, 151, 148,  72, 310,  93,  94,  71,  43,  39, 170,  37,
        68,  54, 107, 398, 160,  30, 183, 315,  45, 121,  38,  82, 134,
        33, 233, 100,  44, 109,  59,  74,  61, 104,  66,  67, 273, 192,
        50, 118, 372,  48, 232, 165,  91, 123,  88, 538,  49,  16, 105,
       536, 216, 425, 117, 551,  99,  86, 101, 394,  78,  95, 181,  76,
        34, 124, 548, 108,  85, 145,  79,  97,  75,  52, 215,  84,  96,
        81, 139, 267,  80, 137, 119, 474, 158, 293, 122, 112, 149, 363,
       271,  31, 171,  87, 168, 176, 347, 163, 190, 486, 133, 302, 268,
       407, 116,  92, 115, 261])

In [96]:
app_val=data['app'].value_counts()>2000
app_val

3       True
12      True
2       True
9       True
15      True
       ...  
407    False
145    False
271    False
398    False
92     False
Name: app, Length: 161, dtype: bool

In [97]:
app_new=app_val[app_val==True]
app_new

3     True
12    True
2     True
9     True
15    True
18    True
14    True
1     True
13    True
8     True
Name: app, dtype: bool

4. By using Pandas derive new features such as - 'day_of_week' , 'day_of_year' , 'month' , and 'hour' as float/int datatypes using the 'click_time' column . Add the newly derived columns in original dataset.

In [98]:
def timefeatures(data):
  data['datetime']=pd.to_datetime(data['click_time'])
  data['day_of_week']=data['datetime'].dt.dayofweek
  data['day_of_year']=data['datetime'].dt.dayofyear
  data['month']=data['datetime'].dt.month
  data['hour']=data['datetime'].dt.hour
  return data

In [99]:
data=timefeatures(data)
data.drop(['click_time','datetime'],axis=1,inplace=True)
data.drop(['attributed_time'],axis=1,inplace=True)
data.head()

,ip,app,device,os,channel,is_attributed,day_of_week,day_of_year,month,hour
0,87540,12,1,13,497,0,1,311,11,9
1,105560,25,1,17,259,0,1,311,11,13
2,101424,12,1,19,212,0,1,311,11,18
3,94584,13,1,13,477,0,1,311,11,4
4,68413,12,1,1,178,0,3,313,11,9


5. Divide the data into training and testing subsets into 80:20 ratio(Train_data = 80% , Testing_data = 20%) and check the average download rates('is_attributed') for train and test data, scores should be comparable.

In [100]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
X=data.drop(['is_attributed'],axis=1)
Y=data['is_attributed']
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(80000, 9)
(80000,)
(20000, 9)
(20000,)


In [101]:
## applying bagging classifier algorithm
from sklearn.ensemble import BaggingClassifier

bgcl = BaggingClassifier(n_estimators=10, max_samples= .7, bootstrap=True)
bgcl = bgcl.fit(X_train, Y_train)
test_pred = bgcl.predict(X_test)
bgcl_score=bgcl.score(X_test, Y_test)
print("The Bagging classifier model accuracy is {}".format(bgcl_score))
print("The confusion matrix of Bagging classifier model is")
print(metrics.confusion_matrix(Y_test,test_pred))

The Bagging classifier model accuracy is 0.9975
The confusion matrix of Bagging classifier model is
[[19943    14]
 [   36     7]]


In [102]:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
xgbl=XGBClassifier()
xgbl.fit(X_train,Y_train)
test_pred=xgbl.predict(X_test)
xgbl_score=xgbl.score(X_test,Y_test)
print("The XGBoostclassifier model accuracy is {}".format(xgbl_score))
print("The confusion matrix of XGBoostclassifier model is")
print(metrics.confusion_matrix(Y_test,test_pred))


The XGBoostclassifier model accuracy is 0.99785
The confusion matrix of XGBoostclassifier model is
[[19956     1]
 [   42     1]]


In [103]:
print('The accuracy for BaggingClassifier is {}'.format(bgcl_score))
print('The accuracy for XGBoostClassifier is {}'.format(xgbl_score))

The accuracy for BaggingClassifier is 0.9975
The accuracy for XGBoostClassifier is 0.99785
